In [ ]:
import numpy
import pandas as pd
import folium
import socket; import struct

In [ ]:
db = pd.read_csv('data/IP2LOCATION-LITE-DB5.CSV')

In [ ]:
db.columns = ['시작주소', '끝주소', '국가코드', '국가명', '시도', '시군구', '위도', '경도']

In [ ]:
db.shape

In [ ]:
db.dtypes

In [ ]:
db.head()

In [ ]:
def ip2int(ip_addr):
    return struct.unpack('!I', socket.inet_aton(ip_addr))[0]

In [ ]:
ip = ip2int('182.252.9.227')
loc = db[(db['시작주소'] <= ip) & (db['끝주소']>=ip)]
loc['위도'].values[0]

In [ ]:
map = folium.Map(location=[loc['위도'].values[0], loc['경도'].values[0]])

folium.Marker(
    location=[loc['위도'].values[0], loc['경도'].values[0]],
    popup='203.245.160.72'
).add_to(map)

map


In [ ]:
dscweb_ips = pd.read_csv('data/dscweb_ip.csv', sep='\t')
dscweb_ips.head()

In [ ]:
dscweb_ips['hits'] = dscweb_ips['hits'].str.replace(',','').astype(int)

In [ ]:
dscweb_ips['ip_int'] = dscweb_ips['ip'].apply(ip2int)
#dscweb_ips = dscweb_ips[dscweb_ips['ip_int']<1000]

In [ ]:
def find_lat(ip):
    found = db[(db['시작주소'] <= ip) & (db['끝주소'] >= ip)]
    return found['위도'].values[0]

def find_alt(ip):
    found = db[(db['시작주소'] <= ip) & (db['끝주소'] >= ip)]
    return found['경도'].values[0]

In [ ]:
dscweb_ips['위도'] = dscweb_ips['ip_int'].apply(find_lat)
dscweb_ips['경도'] = dscweb_ips['ip_int'].apply(find_alt)

In [ ]:
dscweb_ips.dtypes

In [ ]:
map = folium.Map(location=[dscweb_ips['위도'].median(), dscweb_ips['경도'].median()])

for n in dscweb_ips.index:
    folium.Marker(
        location=[dscweb_ips.loc[n,'위도'], dscweb_ips.loc[n, '경도']]
    ).add_to(map)

map

In [ ]:
dscweb_ips.head()

In [ ]:
dscweb_ips.columns

In [ ]:
from plotnine import *
import matplotlib as mpl
mpl.rcParams['font.family']='D2Coding'

In [ ]:
(ggplot(dscweb_ips)
 + aes(x='위도', y='경도', color='ip', size='hits')
 + geom_point()
)